# Music structure boundary estimation from low-rank Laplacian approximations
Music Information Retrieval, Final Project, Christos Plachouras

In [ ]:
import librosa  # for audio processing
import librosa.display  # for plotting
import numpy as np  # for matrix processing
import scipy  # for matrix processing
from scipy.spatial.distance import squareform, pdist  # for eigenvector set distances
import sklearn.cluster  # for spectral clustering

In [ ]:
"""
Segmentation code derivative of:
     https://github.com/bmcfee/lsd_viz (MIT license)
with subsequent modifications from:
    https://github.com/chrispla/hierarchical_structure (MIT license)
"""

def compute_laplacian(path, bins_per_octave, n_octaves):
    """Compute the Laplacian matrix from an audio file using
    its Constant-Q Transform.

    Args:
        path: filepath (str)
        bins_per_octave: number of bins per octave for CQT calculation
        n_octaves: number of octaves for CQT calculation

    Returns:
        y: audio samples (np.array) - returned for plotting
        C: Constant-Q Transform (np.array) - returned for plotting
        C_sync: beat-synchronized CQT (np.array) - returned for plotting
        beats: beat track - returned for formatting segments
        L: Laplacian matrix (np.array)
        
    """

    # load audio
    y, sr = librosa.load(path, sr=22050, mono=True)

    # Compute Constant-Q Transform in dB
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y,
                                                   sr=sr,
                                                   bins_per_octave=bins_per_octave,
                                                   n_bins=n_octaves * bins_per_octave,
                                                   hop_length=512)),
                                                   ref=np.max)

    # beat tracking
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    # get beat times in seconds
    beat_times = librosa.frames_to_time(librosa.util.fix_frames(beats, x_min=0, x_max=C.shape[1]), sr=sr)
    # beats = beats[0::4]  # hard-coded
    # beat_times = beat_times[0::4]  # hard-coded

    # beat synchronize the CQT
    Csync = librosa.util.sync(C, beats, aggregate=np.median)

    # stack 4 consecutive frames
    # Cstack = librosa.feature.stack_memory(Csync, n_steps=4)

    # compute weighted recurrence matrix
    R = librosa.segment.recurrence_matrix(Csync, width=3, mode='affinity', sym=True)

    # enhance diagonals with a median filter
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    Rf = librosa.segment.path_enhance(Rf, 15)

    # compute MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512)
    # beat synchronize them
    Msync = librosa.util.sync(mfcc, beats)

    # build the MFCC sequence matrix
    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)
    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)

    # get the balanced combination of the MFCC sequence matric and the CQT
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)
    A = mu * Rf + (1 - mu) * R_path

    # compute the normalized Laplacian
    L = scipy.sparse.csgraph.laplacian(A, normed=True)

    return y, C, Csync, beats, L


def segment_laplacian(L, k1, k2):
    """Decompose Laplacian and make sets of its first integer k (k1 or k2) eigenvectors.
    For each set, compute its Euclidean self distance over time, and do spectral clustering.

    Args
        L: Laplacian matrix (np.array)
        k1: eigenvectors for first set
        k2: eigenvectors for second set

    Returns:
        distances: self distance matrix of each set of first eigenvectors (np.array, shape=(kmax-kmin, beats, beats))
        segments: estimated structural segments for each set of first eigenvectors (np.array, shape=(kmax-kmin, beats))
    """

    # eigendecomposition
    evals, evecs = scipy.linalg.eigh(L)

    # eigenvector filtering
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))

    # normalization
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5

    # initialize set
    segments = []
    distances = []

    for k in [k1, k2]:
        # create set using first k normalized eigenvectors
        Xs = evecs[:, :k] / Cnorm[:, k-1:k]

        # get eigenvector set distances
        distance = squareform(pdist(Xs, metric='euclidean'))
        distances.append(distance)

        # cluster each set k using k clusters to get k segment types
        KM = sklearn.cluster.KMeans(n_clusters=k, n_init=50, max_iter=500)
        segments.append(KM.fit_predict(Xs))

    return np.asarray(segments)